In [194]:
import dash
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import numpy as np
import sqlalchemy as sqla

In [195]:
import pycountry
def alpha3(country):
    try:
        return pycountry.countries.get(name=country).alpha_3
    except:
        pass
        
countries = pd.read_csv("../data/countriesdata.csv").drop(columns=["Unnamed: 0"])
countries["code3"] = countries['country'].apply(lambda x: alpha3(x))

countries.loc[2, "code3"]="BOL"

countries['color']=range(19)

In [196]:
def listunique(list):
    unique = []
    for i in list:
        if i not in unique:
            unique.append(i)
    return unique

In [197]:
##Setting Date Options
import datetime
start_date = datetime.date(2017, 1, 6)
end_date = datetime.date(2021, 1, 1)
delta = datetime.timedelta(days = 7)
ls_date = []
while start_date <= end_date:
    ls_date.append(str(start_date))
    start_date += delta

In [198]:
##App Layout
app = JupyterDash(__name__)

periods = {
    'Weekly': ls_date[:-1],
    'Monthly': listunique([x[:-3] for x in ls_date][:-1]),
    'Yearly': ["2017", "2018", "2019", "2020"]
}

app.layout = html.Div([
    html.H1("Most Popular Music in the Americas (2017-2020)", style={'text-align':'center'}),
    dcc.Dropdown(id="Select_Type",
                options=[
                    {"label":"Song","value":"Song"},
                    {"label":"Artist","value":"Artist"},
                    {"label":"Genre","value":"Genre"}],
                placeholder="Choose a Type (Song, Artist, Genre)"
                ),
    dcc.Dropdown(id="Select_Period",
                options=[
                    {"label":"Weekly","value":"Weekly"},
                    {"label":"Monthly","value":"Monthly"},
                    {"label":"Yearly","value":"Yearly"}],
                placeholder="Choose Date by (Weekly, Monthly, Yearly)"
                ),
    dcc.Dropdown(id="Select_Date",
                placeholder="Choose a Daterange"
                ),
    html.Div(id='Loading Status'),
    dcc.Graph(id='Map',style={'width': '100%'})


    ])

##Update Options
@app.callback(
    dash.dependencies.Output(component_id="Select_Date", component_property="options"),
    dash.dependencies.Input(component_id="Select_Period", component_property="value")
    )
def update_options(value):
    return [{'label': i, 'value': i} for i in periods[value]]
    
##Plot Map
@app.callback(
    [Output(component_id="Map", component_property="figure"),
     Output(component_id="Loading Status", component_property="children")],
    [Input(component_id="Select_Type", component_property="value"),
    Input(component_id="Select_Period", component_property="value"),
    Input(component_id="Select_Date", component_property="value")]
    )
def update_figure(type_value,period_value,date_value):
    figdata = countries.copy()
    figdata["value"]=""
    figdata["value"].astype("object")
    ##Weekly, Songs
    if type_value=='Song' and period_value=='Weekly':
        for i, code in enumerate(figdata['code']):
            figdata.at[i,'value'] = pd.read_sql('SELECT song FROM top200data WHERE id LIKE "{}%{}" AND rank in (1,2,3) AND song IS NOT NULL'\
                                                 .format(date_value, code),engine).values
    ##Monthly/Yearly, Songs
    if type_value=='Song' and (period_value=='Monthly' or 'Yearly'):
        for i, code in enumerate(figdata['code']):
            monthly = pd.read_sql('SELECT * FROM top200data WHERE date like "{}%" AND regioncode = "{}" AND song IS NOT NULL'\
                                  .format(date_value, code) ,engine)
            monthly['streams'] = monthly['streams'].apply(lambda x: int(x.replace(",","")))
            figdata.at[i,'value'] = monthly.groupby(["song","songcode","regioncode"]).streams.sum().sort_values(ascending=False)[0:3].keys().get_level_values(0)
    
    ##Artist
    if type_value=='Artist':
        for i, code in enumerate(figdata['code']):
            artist = pd.read_sql('SELECT * FROM top200data WHERE date like "{}%" AND regioncode = "{}" AND artist IS NOT NULL'\
                                  .format(date_value, code) ,engine)
            artist['streams'] = artist['streams'].apply(lambda x: int(x.replace(",","")))
            artist['artist'] = artist['artist'].apply(lambda x: x.split(', ')[0])
            figdata.at[i,'value'] = artist.groupby(["artist","regioncode"]).streams.sum().sort_values(ascending=False)[0:3].keys().get_level_values(0)

    ##Genre
    if type_value=='Genre':
        for i, code in enumerate(figdata['code']): 
            artist = pd.read_sql('SELECT * FROM top200data WHERE date like "{}%" AND regioncode = "{}" AND artist IS NOT NULL'\
                                  .format(date_value, code) ,engine)
            artist['streams'] = artist['streams'].apply(lambda x: int(x.replace(",","")))
            artist['artist'] = artist['artist'].apply(lambda x: x.split(', ')[0])
            genre = artist.copy().groupby(["artist","regioncode"]).streams.sum().to_frame().merge(pd.read_csv('../data/artistsdata.csv'), left_on="artist", right_on="artist").dropna()
            genre['genre'] = genre['genre'].apply(lambda x: str(x)[1:-1].replace("'","").split(", "))
            figdata.at[i,'value'] = genre.explode('genre').groupby('genre').streams.sum().sort_values(ascending=False)[0:3].keys()    
            
    fig = go.Figure(data=go.Choropleth(
        locations = figdata['code3'],
        z = figdata['color'],
        text = figdata['value'],
        customdata = figdata['code3'],
        hovertemplate = '1. %{text[0]}<br>2. %{text[1]}<br>3. %{text[2]}<extra>%{customdata}</extra>',
        colorscale = 'Rainbow',
        autocolorscale = False,
        reversescale = True,
        showscale = False
    ))

    fig.update_geos(fitbounds="locations")
    fig.update_layout(
        title_text='',
        geo=dict(
            showframe=True,
            showcoastlines=False,
            projection_type='equirectangular'
        ),
        annotations = [dict(
            x=0.55,
            y=0.1,
            xref='paper',
            yref='paper',
            text='Source: <a href="https://developer.spotify.com">\
                Spotify</a>',
            showarrow = False
        )]
    )
    status = "The {} Data for Top 3 {}s of {} has been loaded.".format(period_value, type_value, date_value)
    return fig, status


app.run_server()

OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.

In [9]:
##Weekly Songs
engine = sqla.create_engine('sqlite:///top200data')
pd.read_sql('SELECT * FROM top200data WHERE date = "2017-01-06" AND regioncode = "us" AND rank IN (1,2,3) ',engine)
pd.read_sql('SELECT song FROM top200data WHERE id LIKE "{}%{}" AND rank in (1,2,3)'.format("2017-01-06","us"),engine).values[0]
for i, code in enumerate(countries['code']):
    countries.at[i,'song'] = pd.read_sql('SELECT song FROM top200data WHERE id LIKE "{}%{}" AND rank in (1,2,3)'.format("2017-01-06", code),engine).values

In [10]:
##Monthly Songs
# monthly = pd.read_sql('SELECT * FROM top200data WHERE date like "2017-01%" and regioncode  ',engine)
# monthly['streams'] = monthly['streams'].apply(lambda x: int(x.replace(",","")))
figdata.at[0,'song'] = np.array(monthly.groupby(["song","songcode","regioncode"]).streams.sum().sort_values(ascending=False)[0:3].keys().get_level_values(0).tolist()).astype('str')

NameError: name 'monthly' is not defined

In [17]:
figdata=countries.copy()
for i, code in enumerate(figdata['code']):
    monthly = pd.read_sql('SELECT * FROM top200data WHERE date like "{}%" AND regioncode = "{}" '\
                          .format('2019-02', code) ,engine)
    monthly['streams'] = monthly['streams'].apply(lambda x: int(x.replace(",","")))
    figdata.at[i,'song'] = monthly.groupby(["song","songcode","regioncode"]).streams.sum().sort_values(ascending=False)[0:3].keys().get_level_values(0)[0]


In [72]:
##Artist
figdata=countries.copy()
figdata['artist']=""
figdata.astype("object")
for i, code in enumerate(figdata['code']):
    artist = pd.read_sql('SELECT * FROM top200data WHERE date like "{}%" AND regioncode = "{}" AND artist IS NOT NULL'\
                          .format('2017-02', code) ,engine)
    artist['streams'] = artist['streams'].apply(lambda x: int(x.replace(",","")))
    artist['artist'] = artist['artist'].apply(lambda x: x.split(', ')[0])
    figdata.at[i,'artist'] = artist.groupby(["artist","regioncode"]).streams.sum().sort_values(ascending=False)[0:3].keys().get_level_values(0)

In [184]:
##Genre
if type_value=='Artist':
    for i, code in enumerate(figdata['code']):
        artist = pd.read_sql('SELECT * FROM top200data WHERE date like "{}%" AND regioncode = "{}" AND artist IS NOT NULL'\
                              .format('2020', 'us') ,engine)
        artist['streams'] = artist['streams'].apply(lambda x: int(x.replace(",","")))
        artist['artist'] = artist['artist'].apply(lambda x: x.split(', ')[0])
        figdata.at[i,'value'] = artist.groupby(["artist","regioncode"]).streams.sum().sort_values(ascending=False)[0:3].keys().get_level_values(0)


NameError: name 'type_value' is not defined

In [188]:
def clean_alt_list(list_):
    list_ = list_.replace(', ', '","')
    list_ = list_.replace('[', '["')
    list_ = list_.replace(']', '"]')
    return list_
artist = pd.read_sql('SELECT * FROM top200data WHERE date like "{}%" AND regioncode = "{}" AND artist IS NOT NULL'\
                              .format('2020', 'us') ,engine)
artist['streams'] = artist['streams'].apply(lambda x: int(x.replace(",","")))
artist['artist'] = artist['artist'].apply(lambda x: x.split(', ')[0])
genre = artist.groupby(["artist","regioncode"]).streams.sum().to_frame().merge(pd.read_csv('../data/artistsdata.csv'), left_on="artist", right_on="artist").dropna()
genre['genre'][1][1:-1].replace("'","").split(", ")
genre['genre'] = genre['genre'].apply(lambda x: str(x)[1:-1].replace("'","").split(", "))
genre.explode('genre').groupby('genre').streams.sum().sort_values(ascending=False)[0:3].keys()

Index(['rap', 'pop', 'melodic rap'], dtype='object', name='genre')